# Imports 

In [3]:
!pip install tables==3.6.1
!pip install socceraction==0.2.0

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
tensorflow 1.9.0 has requirement setuptools<=39.1.0, but you'll have setuptools 39.2.0 which is incompatible.
spacy 2.0.13 has requirement msgpack-numpy<0.29,<0.4.4.0murmurhash>=0.28, but you'll have msgpack-numpy 0.4.4.3 which is incompatible.
spacy 2.0.13 has requirement regex==2018.01.10, but you'll have regex 2017.11.9 which is incompatible.
keras 2.2.2 has requirement keras_applications==1.0.4, but you'll have keras-applications 1.0.2 which is incompatible.
keras 2.2.2 has requirement keras_preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.1 which is incompatible.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Running setup.py bdist_wheel for socceraction: started
  Running setup.py bdist_wheel for socceraction: finished with status 'done'
  Stored in directory: C:\Users\Anuj\AppData\Local\pip\Cache\wheels\24\91\85\1211f86b475fba73e483657a89347d82cc15d261847a85995d
Successfully built socceraction
  Found existing installation: socceraction 0.2.1
    Uninstalling socceraction-0.2.1:
      Successfully uninstalled socceraction-0.2.1


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
tensorflow 1.9.0 has requirement setuptools<=39.1.0, but you'll have setuptools 39.2.0 which is incompatible.
spacy 2.0.13 has requirement msgpack-numpy<0.29,<0.4.4.0murmurhash>=0.28, but you'll have msgpack-numpy 0.4.4.3 which is incompatible.
spacy 2.0.13 has requirement regex==2018.01.10, but you'll have regex 2017.11.9 which is incompatible.
keras 2.2.2 has requirement keras_applications==1.0.4, but you'll have keras-applications 1.0.2 which is incompatible.
keras 2.2.2 has requirement keras_preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.1 which is incompatible.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Ellipse
import seaborn as sns
from math import *
import matplotlib.pylab as pyl
import pickle
import swifter
import warnings
import plotly.express as px
from itertools import chain
import scipy.stats as sps
from tqdm import tqdm
from unidecode import unidecode
import regex as re
from io import BytesIO
from pathlib import Path
from tqdm.notebook import tqdm
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile
import pandas as pd
from sklearn.metrics import brier_score_loss, roc_auc_score  # version 0.22.2
from xgboost import XGBClassifier  # version 1.0.2

import socceraction.vaep.features as features
import socceraction.vaep.labels as labels

from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value

C:\Users\Anuj\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#pd.set_option('max_colwidth', 999)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_rows", 3000)

In [3]:
import warnings
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

# Preprocess the data

## Preprocess the Wyscout data

In [7]:
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')

### Teams

In [8]:
json_teams = read_json_file('teams.json')
df_teams = pd.read_json(json_teams)

In [16]:
df_teams.head(10)

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alavés,696,Deportivo Alavés,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
5,Troyes,Troyes,3795,Espérance Sportive Troyes Aube Champagne,"{'name': 'France', 'id': '250', 'alpha3code': ...",club
6,Getafe (Madrid),Getafe,698,Getafe Club de Fútbol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
7,Mönchengladbach,Borussia M'gladbach,2454,Borussia VfL Mönchengladbach,"{'name': 'Germany', 'id': '276', 'alpha3code':...",club
8,"Huddersfield, West Yorkshire",Huddersfield Town,1673,Huddersfield Town FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
9,Bilbao,Athletic Club,678,Athletic Club Bilbao,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club


In [9]:
df_teams.to_hdf('wyscout.h5', key='teams', mode='w')

### Players

In [10]:
json_players = read_json_file('players.json')
df_players = pd.read_json(json_players)

In [19]:
df_players.head(10)

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konaté,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konaté,null
5,"{'name': 'Netherlands', 'id': '528', 'alpha3co...",83,Jasper,,Cillessen,676,1989-04-22,185,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Netherlands', 'id': '528', 'alpha3co...",33,right,J. Cillessen,664
6,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",91,Toby,,Alderweireld,1624,1989-03-02,187,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",36,right,T. Alderweireld,5629
7,"{'name': 'Belgium', 'id': '56', 'alpha3code': ...",88,Jan,,Vertonghen,1624,1987-04-24,189,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'Belgium', 'id': '56', 'alpha3code': ...",48,left,J. Vertonghen,5629
8,"{'name': 'France', 'id': '250', 'alpha3code': ...",74,Alexander,,Djiku,3783,1994-08-09,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",229427,right,A. Djiku,null
9,"{'name': 'Denmark', 'id': '208', 'alpha3code':...",76,Christian,,Dannemann Eriksen,1624,1992-02-14,180,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'Denmark', 'id': '208', 'alpha3code':...",54,right,C. Eriksen,7712


In [11]:
df_players.to_hdf('wyscout.h5', key='players', mode='a')

### Matches

In [12]:
competitions = [
     'England',
#     'France',
#     'Germany',
#     'Italy',
#     'Spain',
#    'European Championship',
#     'World Cup'
]

In [13]:
dfs_matches = []
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_matches = f'matches_{competition_name}.json'
    json_matches = read_json_file(file_matches)
    df_matches = pd.read_json(json_matches)
    dfs_matches.append(df_matches)
df_matches = pd.concat(dfs_matches)

In [23]:
df_matches.head(10)

,status,roundId,gameweek,teamsData,seasonId,dateutc,winner,venue,wyId,label,date,referees,duration,competitionId
0,Played,4405654,38,"{'1646': {'scoreET': 0, 'coachId': 8880, 'side...",181150,2018-05-13 14:00:00,1659,Turf Moor,2500089,"Burnley - AFC Bournemouth, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385705, 'role': 'referee'}, {'r...",Regular,364
1,Played,4405654,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",181150,2018-05-13 14:00:00,1628,Selhurst Park,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381851, 'role': 'referee'}, {'r...",Regular,364
2,Played,4405654,38,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",181150,2018-05-13 14:00:00,1609,The John Smith's Stadium,2500091,"Huddersfield Town - Arsenal, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384965, 'role': 'referee'}, {'r...",Regular,364
3,Played,4405654,38,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",181150,2018-05-13 14:00:00,1612,Anfield,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385704, 'role': 'referee'}, {'r...",Regular,364
4,Played,4405654,38,"{'1644': {'scoreET': 0, 'coachId': 93112, 'sid...",181150,2018-05-13 14:00:00,1611,Old Trafford,2500093,"Manchester United - Watford, 1 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 381853, 'role': 'referee'}, {'r...",Regular,364
5,Played,4405654,38,"{'1613': {'scoreET': 0, 'coachId': 210700, 'si...",181150,2018-05-13 14:00:00,1613,St. James' Park,2500094,"Newcastle United - Chelsea, 3 - 0","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 384888, 'role': 'referee'}, {'r...",Regular,364
6,Played,4405654,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",181150,2018-05-13 14:00:00,1625,St. Mary's Stadium,2500095,"Southampton - Manchester City, 0 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 385911, 'role': 'referee'}, {'r...",Regular,364
7,Played,4405654,38,"{'10531': {'scoreET': 0, 'coachId': 32573, 'si...",181150,2018-05-13 14:00:00,1639,Liberty Stadium,2500096,"Swansea City - Stoke City, 1 - 2","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378952, 'role': 'referee'}, {'r...",Regular,364
8,Played,4405654,38,"{'1631': {'scoreET': 0, 'coachId': 209010, 'si...",181150,2018-05-13 14:00:00,1624,Wembley Stadium,2500097,"Tottenham Hotspur - Leicester City, 5 - 4","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 378951, 'role': 'referee'}, {'r...",Regular,364
9,Played,4405654,38,"{'1623': {'scoreET': 0, 'coachId': 8541, 'side...",181150,2018-05-13 14:00:00,1633,London Stadium,2500098,"West Ham United - Everton, 3 - 1","May 13, 2018 at 4:00:00 PM GMT+2","[{'refereeId': 408156, 'role': 'referee'}, {'r...",Regular,364


In [14]:
df_matches.to_hdf('wyscout.h5', key='matches', mode='a')

### Events

In [15]:
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_events = f'events_{competition_name}.json'
    json_events = read_json_file(file_events)
    df_events = pd.read_json(json_events)
    df_events_matches = df_events.groupby('matchId', as_index=False)
    for match_id, df_events_match in df_events_matches:
        df_events_match.to_hdf('wyscout.h5', key=f'events/match_{match_id}', mode='a')

## Convert the Wyscout data to the SPADL representation

In [16]:
convert_to_spadl('wyscout.h5', 'spadl.h5')

  0%|                                                                                        | 0/380 [00:00<?, ?game/s]

...Inserting actiontypes
...Inserting bodyparts
...Inserting results
...Converting games
...Converting players
...Converting teams
...Generating player_games


  0%|                                                                                        | 0/380 [00:00<?, ?game/s]

...Converting events to actions


100%|██████████████████████████████████████████████████████████████████████████████| 380/380 [07:52<00:00,  1.24s/game]


# Value game states

In [17]:
df_games = pd.read_hdf('spadl.h5', key='games')
df_actiontypes = pd.read_hdf('spadl.h5', key='actiontypes')
df_bodyparts = pd.read_hdf('spadl.h5', key='bodyparts')
df_results = pd.read_hdf('spadl.h5', key='results')

In [19]:
nb_prev_actions = 3

## Generate game state features

In [20]:
functions_features = [
    features.actiontype_onehot,
    features.bodypart_onehot,
    features.result_onehot,
    features.goalscore,
    features.startlocation,
    features.endlocation,
    features.movement,
    features.space_delta,
    features.startpolar,
    features.endpolar,
    features.team,
    features.time_delta
]

In [21]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    dfs_gamestates = features.gamestates(df_actions, nb_prev_actions=nb_prev_actions)
    dfs_gamestates = features.play_left_to_right(dfs_gamestates, game['home_team_id'])
    
    df_features = pd.concat([function(dfs_gamestates) for function in functions_features], axis=1)
    df_features.to_hdf('features.h5', key=f'game_{game_id}')

## Generate game state labels

The following cell lists a number of *label generators* from the `labels` module in the `socceraction` library. Each function expects either a `DataFrame` object containing actions (i.e., individual actions) or a list of `DataFrame` objects containing consecutive actions (i.e., game states), and returns the corresponding *label* for the individual action or game state.

In [22]:
functions_labels = [
    labels.scores,
    labels.concedes
]

The following cell computes the labels for each action:

1. Obtain the actions for the game (i.e., `df_actions`) by looping through the games;
2. Compute the labels for the actions (i.e., `df_labels`) by looping through the list of *label generators*.

In [23]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    df_labels = pd.concat([function(df_actions) for function in functions_labels], axis=1)
    df_labels.to_hdf('labels.h5', key=f'game_{game_id}')

## Generate dataset

In [24]:
columns_features = features.feature_column_names(functions_features, nb_prev_actions=nb_prev_actions)

In [25]:
dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    dfs_features.append(df_features[columns_features])
df_features = pd.concat(dfs_features).reset_index(drop=True)

In [26]:
df_features.head(10)

,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,type_take_on_a0,type_foul_a0,type_tackle_a0,...,end_dist_to_goal_a0,end_angle_to_goal_a0,end_dist_to_goal_a1,end_angle_to_goal_a1,end_dist_to_goal_a2,end_angle_to_goal_a2,team_1,team_2,time_delta_1,time_delta_2
0,True,False,False,False,False,False,False,False,False,False,...,63.091679,0.053916,63.091679,0.053916,63.091679,0.053916,True,True,0.000000,0.000000
1,True,False,False,False,False,False,False,False,False,False,...,68.328929,0.355773,63.091679,0.053916,63.091679,0.053916,True,True,1.997756,1.997756
2,True,False,False,False,False,False,False,False,False,False,...,73.715416,0.185556,68.328929,0.355773,63.091679,0.053916,True,True,0.771744,2.769500
3,True,False,False,False,False,False,False,False,False,False,...,38.707772,0.396818,73.715416,0.185556,68.328929,0.355773,True,True,2.174464,2.946208
4,True,False,False,False,False,False,False,False,False,False,...,37.425928,0.620467,38.707772,0.396818,73.715416,0.185556,True,True,3.907382,6.081846
5,True,False,False,False,False,False,False,False,False,False,...,30.729141,1.258205,37.425928,0.620467,38.707772,0.396818,True,True,3.758730,7.666112
6,False,True,False,False,False,False,False,False,False,False,...,8.400000,0.000000,30.729141,1.258205,37.425928,0.620467,True,True,2.210584,5.969314
7,False,False,False,False,False,False,False,False,False,False,...,106.094842,0.246030,96.600000,0.000000,99.923872,0.296969,False,False,1.756122,3.966706
8,False,True,False,False,False,False,False,False,False,False,...,25.925192,1.489705,25.925192,1.489705,8.400000,0.000000,False,True,2.095783,3.851905
9,False,False,False,False,False,False,False,False,False,False,...,89.647658,0.388999,106.094842,0.246030,106.094842,0.246030,False,True,3.034782,5.130565


In [27]:
columns_labels = [
    'scores',
    'concedes'
]

In [28]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    dfs_labels.append(df_labels[columns_labels])
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

In [29]:
df_labels.head(10)

,scores,concedes
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


## Train classifiers

In [30]:
models = {}
for column_labels in columns_labels:
    model = XGBClassifier(n_estimators=100, max_depth=4)
    model.fit(df_features, df_labels[column_labels])
    models[column_labels] = model

## Estimate probabilities

In [31]:
dfs_predictions = {}
for column_labels in columns_labels:
    model = models[column_labels]
    probabilities = model.predict_proba(df_features)
    predictions = probabilities[:, 1]
    dfs_predictions[column_labels] = pd.Series(predictions)
df_predictions = pd.concat(dfs_predictions, axis=1)

In [32]:
df_predictions.head(10)

,scores,concedes
0,0.005669,0.002083
1,0.006183,0.001688
2,0.005846,0.001996
3,0.012178,0.002300
4,0.015808,0.001493
5,0.018279,0.001615
6,0.013081,0.003892
7,0.002941,0.039347
8,0.012118,0.003834
9,0.002360,0.012086


The following cell obtains the `game_id` for each action in order to store the predictions per game.

In [33]:
dfs_game_ids = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    dfs_game_ids.append(df_actions['game_id'])
df_game_ids = pd.concat(dfs_game_ids, axis=0).astype('int').reset_index(drop=True)

In [34]:
df_predictions = pd.concat([df_predictions, df_game_ids], axis=1)

In [35]:
df_predictions.head(10)

,scores,concedes,game_id
0,0.005669,0.002083,2500089
1,0.006183,0.001688,2500089
2,0.005846,0.001996,2500089
3,0.012178,0.002300,2500089
4,0.015808,0.001493,2500089
5,0.018279,0.001615,2500089
6,0.013081,0.003892,2500089
7,0.002941,0.039347,2500089
8,0.012118,0.003834,2500089
9,0.002360,0.012086,2500089


In [36]:
df_predictions_per_game = df_predictions.groupby('game_id')

In [37]:
for game_id, df_predictions in tqdm(df_predictions_per_game):
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions[columns_labels].to_hdf('predictions.h5', key=f'game_{game_id}')

# Value on-the-ball actions - VAEP Values

<img src="./expl_photo/1.JPG">
<img src="./expl_photo/2.JPG">
<img src="./expl_photo/3.JPG">
<img src="./expl_photo/4.JPG">

In [38]:
df_players = pd.read_hdf('spadl.h5', key='players')
df_teams = pd.read_hdf('spadl.h5', key='teams')

In [39]:
dfs_values = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .merge(df_players, how='left')
        .merge(df_teams, how='left')
        .reset_index(drop=True)
    )
    
    df_predictions = pd.read_hdf('predictions.h5', key=f'game_{game_id}')
    df_values = value(df_actions, df_predictions['scores'], df_predictions['concedes'])
    
    df_all = pd.concat([df_actions, df_predictions, df_values], axis=1)
    dfs_values.append(df_all)

In [40]:
df_values = (pd.concat(dfs_values)
    .sort_values(['game_id', 'period_id', 'time_seconds'])
    .reset_index(drop=True)
)

In [41]:
df_values[
    ['short_name', 'scores', 'concedes', 'offensive_value', 'defensive_value', 'vaep_value']
].head(10)

,short_name,scores,concedes,offensive_value,defensive_value,vaep_value
0,A. Lacazette,0.004934,0.001779,0.000000,-0.000000,0.000000
1,R. Holding,0.008188,0.002175,0.003254,-0.000396,0.002858
2,M. Özil,0.006571,0.001441,-0.001617,0.000735,-0.000883
3,Mohamed Elneny,0.005887,0.001814,-0.000684,-0.000373,-0.001057
4,Bellerín,0.013319,0.001706,0.007433,0.000107,0.007540
5,M. Özil,0.009344,0.003094,-0.003975,-0.001387,-0.005363
6,H. Maguire,0.005380,0.002681,0.002286,0.006663,0.008949
7,Bellerín,0.010335,0.002063,0.007655,0.003317,0.010972
8,Bellerín,0.011106,0.001742,0.000771,0.000320,0.001091
9,G. Xhaka,0.008079,0.001792,-0.003027,-0.000049,-0.003076


In [60]:
df_values

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,type_id,result_id,type_name,result_name,bodypart_name,short_name,first_name,last_name,birth_date,short_team_name,team_name,scores,concedes,offensive_value,defensive_value,vaep_value
0,2499719.0,1.0,2.758649,1609.0,25413.0,51.45,34.68,32.55,14.96,0,0,1,pass,success,foot,A. Lacazette,Alexandre,Lacazette,1991-05-28,Arsenal,Arsenal FC,0.004934,0.001779,0.000000,-0.000000,0.000000
1,2499719.0,1.0,4.946850,1609.0,370224.0,32.55,14.96,53.55,17.00,0,0,1,pass,success,foot,R. Holding,Rob,Holding,1995-09-20,Arsenal,Arsenal FC,0.008188,0.002175,0.003254,-0.000396,0.002858
2,2499719.0,1.0,6.542188,1609.0,3319.0,53.55,17.00,36.75,19.72,1,0,1,pass,success,head,M. Özil,Mesut,Özil,1988-10-15,Arsenal,Arsenal FC,0.006571,0.001441,-0.001617,0.000735,-0.000883
3,2499719.0,1.0,8.143395,1609.0,120339.0,36.75,19.72,43.05,3.40,1,0,1,pass,success,head,Mohamed Elneny,Mohamed Naser,Elsayed Elneny,1992-07-11,Arsenal,Arsenal FC,0.005887,0.001814,-0.000684,-0.000373,-0.001057
4,2499719.0,1.0,10.302366,1609.0,167145.0,43.05,3.40,75.60,8.16,0,0,1,pass,success,foot,Bellerín,Héctor,Bellerín Moruno,1995-03-19,Arsenal,Arsenal FC,0.013319,0.001706,0.007433,0.000107,0.007540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483660,2500098.0,2.0,2785.702032,1633.0,20620.0,55.65,7.48,103.95,19.04,0,0,1,pass,success,foot,Pedro Obiang,Pedro,Mba Obiang Avomo,1992-03-27,West Ham United,West Ham United FC,0.077056,0.001544,0.064642,0.000003,0.064645
483661,2500098.0,2.0,2789.232666,1633.0,14703.0,103.95,19.04,103.95,19.04,0,1,0,cross,fail,foot,M. Arnautović,Marko,Arnautović,1989-04-19,West Ham United,West Ham United FC,0.026961,0.003217,-0.050096,-0.001673,-0.051768
483662,2500098.0,2.0,2790.457942,1623.0,8239.0,102.90,21.08,105.00,21.76,0,10,1,interception,success,foot,L. Baines,Leighton,Baines,1984-12-11,Everton,Everton FC,0.005487,0.082486,0.002271,-0.055525,-0.053254
483663,2500098.0,2.0,2829.821084,1633.0,70965.0,105.00,0.00,92.40,36.04,0,5,1,corner_crossed,success,foot,João Mário,João Mário,Naval da Costa Eduardo,1993-01-19,West Ham United,West Ham United FC,0.094044,0.002573,0.047544,-0.002573,0.044970


* scores = P_scores(Si, x) - prob(scoring in the next 10 actions) - obtained from the ML Model - given for the particular `state`
* concedes = P_concedes(Si, x) - prob(concedes in the next 10 actions) - obtained from the ML Model - given for the particular `state`
* offensive_value = P_scores(Si) - P_scores(Si-1) - given for the particular `action`
* defensive_value = P_concedes(Si-1) - P_concedes(Si) - given for the particular `action`

In [64]:
df_values.to_pickle('./vaep_values.pkl')

# VAEP for Defenders 

## Pre-Processing 

In [88]:
vaep_values = pd.read_pickle('./vaep_values.pkl')

In [89]:
vaep_values = vaep_values.replace({'short_team_name':{
        'Manchester United': 'Man Utd',
        'Tottenham Hotspur': 'Spurs',
        'West Ham United': 'West Ham',
        'Manchester City': 'Man City',
        'Brighton & Hove Albion': 'Brighton',
        'Stoke City': 'Stoke',
        'AFC Bournemouth': 'Bournemouth',
        'West Bromwich Albion': 'West Brom',
        'Leicester City': 'Leicester',
        'Swansea City': 'Swansea',
        'Huddersfield Town': 'Huddersfield',
        'Newcastle United': 'Newcastle'
    }}
)

vaep_values['game_id'] = vaep_values['game_id'].astype(int)
vaep_values['temp'] = vaep_values['game_id'].astype(str) + vaep_values['short_team_name']
match_def['temp'] = match_def['wyId'].astype(str) + match_def['team']

In [90]:
vaep_values = vaep_values.merge(match_def[['temp', 'footedness']], left_on='temp', right_on='temp', how='left')
vaep_values.drop(columns = ['temp'], inplace=True)

In [91]:
vaep_values['name'] = vaep_values['first_name']+vaep_values['last_name']
vaep_values['name'] = vaep_values['name'].astype(str).apply(lambda x: unidecode(x))
vaep_values['name'] = vaep_values['name'].apply(lambda x: x.replace('-', ''))
vaep_values['name'] = vaep_values['name'].apply(lambda x: x.replace(' ', ''))
vaep_values

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,bodypart_id,type_id,result_id,type_name,result_name,bodypart_name,short_name,first_name,last_name,birth_date,short_team_name,team_name,scores,concedes,offensive_value,defensive_value,vaep_value,footedness,name
0,2499719,1.0,2.758649,1609.0,25413.0,51.45,34.68,32.55,14.96,0,0,1,pass,success,foot,A. Lacazette,Alexandre,Lacazette,1991-05-28,Arsenal,Arsenal FC,0.004934,0.001779,0.000000,-0.000000,0.000000,right-left-left,AlexandreLacazette
1,2499719,1.0,4.946850,1609.0,370224.0,32.55,14.96,53.55,17.00,0,0,1,pass,success,foot,R. Holding,Rob,Holding,1995-09-20,Arsenal,Arsenal FC,0.008188,0.002175,0.003254,-0.000396,0.002858,right-left-left,RobHolding
2,2499719,1.0,6.542188,1609.0,3319.0,53.55,17.00,36.75,19.72,1,0,1,pass,success,head,M. Özil,Mesut,Özil,1988-10-15,Arsenal,Arsenal FC,0.006571,0.001441,-0.001617,0.000735,-0.000883,right-left-left,MesutOzil
3,2499719,1.0,8.143395,1609.0,120339.0,36.75,19.72,43.05,3.40,1,0,1,pass,success,head,Mohamed Elneny,Mohamed Naser,Elsayed Elneny,1992-07-11,Arsenal,Arsenal FC,0.005887,0.001814,-0.000684,-0.000373,-0.001057,right-left-left,MohamedNaserElsayedElneny
4,2499719,1.0,10.302366,1609.0,167145.0,43.05,3.40,75.60,8.16,0,0,1,pass,success,foot,Bellerín,Héctor,Bellerín Moruno,1995-03-19,Arsenal,Arsenal FC,0.013319,0.001706,0.007433,0.000107,0.007540,right-left-left,HectorBellerinMoruno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483660,2500098,2.0,2785.702032,1633.0,20620.0,55.65,7.48,103.95,19.04,0,0,1,pass,success,foot,Pedro Obiang,Pedro,Mba Obiang Avomo,1992-03-27,West Ham,West Ham United FC,0.077056,0.001544,0.064642,0.000003,0.064645,right-left-left,PedroMbaObiangAvomo
483661,2500098,2.0,2789.232666,1633.0,14703.0,103.95,19.04,103.95,19.04,0,1,0,cross,fail,foot,M. Arnautović,Marko,Arnautović,1989-04-19,West Ham,West Ham United FC,0.026961,0.003217,-0.050096,-0.001673,-0.051768,right-left-left,MarkoArnautovic
483662,2500098,2.0,2790.457942,1623.0,8239.0,102.90,21.08,105.00,21.76,0,10,1,interception,success,foot,L. Baines,Leighton,Baines,1984-12-11,Everton,Everton FC,0.005487,0.082486,0.002271,-0.055525,-0.053254,right-right-left,LeightonBaines
483663,2500098,2.0,2829.821084,1633.0,70965.0,105.00,0.00,92.40,36.04,0,5,1,corner_crossed,success,foot,João Mário,João Mário,Naval da Costa Eduardo,1993-01-19,West Ham,West Ham United FC,0.094044,0.002573,0.047544,-0.002573,0.044970,right-left-left,JoaoMarioNavaldaCostaEduardo


## MetricsCollection & Clustering

In [81]:
footedness_patterns = match_def["footedness"].unique()

In [82]:
match_def.rename(columns={'R-CB':'R_CB',"L-CB":'L_CB'},inplace=True)

**Filtering out pass data**

In [92]:
vaep_values_pass = vaep_values.loc[vaep_values['type_name']=='pass']

**Creating seperate dataframes for each defensive lineup based for preferred foot of each defender**

In [85]:
df_rrrl = match_def.loc[match_def['footedness']=='right-right-right-left']
df_rrll = match_def.loc[match_def['footedness']=='right-right-left-left']
df_rrl = match_def.loc[match_def['footedness']=='right-right-left']
df_rrr = match_def.loc[match_def['footedness']=='right-right-right']
df_rll = match_def.loc[match_def['footedness']=='right-left-left']
df_rrrll = match_def.loc[match_def['footedness']=='right-right-right-left-left']
df_rrlr = match_def.loc[match_def['footedness']=='right-right-left-right']
df_rrrr = match_def.loc[match_def['footedness']=='right-right-right-right']
df_rrrrl = match_def.loc[match_def['footedness']=='right-right-right-right-left']
df_rlr = match_def.loc[match_def['footedness']=='right-left-right']
df_rrrlr = match_def.loc[match_def['footedness']=='right-right-right-left-right']
df_rrlll = match_def.loc[match_def['footedness']=='right-right-left-left-left']
df_rlll = match_def.loc[match_def['footedness']=='right-left-left-left']

**Creating a list of such dataframes**

In [86]:
df_clusters = [df_rrrl,df_rrll,df_rrl,df_rrr,df_rll,df_rrrll,df_rrlr,df_rrrr,df_rrrrl,df_rlr,df_rrrlr,df_rrlll,df_rlll]

**Creating a dictionary of mapping of players with a mismatch in names in events data and Premier League parsed data**

In [87]:
player_map = {  'RamiroFunesMori': 'JoseRamiroFunesMori',
                'KurtZouma': 'KurtHappyZouma',
                'Danilo': 'DaniloLuizdaSilva',
                'CesarAzpilicueta': 'CesarAzpilicuetaTanco',
                'EzequielSchelotto': 'MatiasEzequielSchelotto',
                'GaetanBong': 'GaetanBongSongo',
                'HectorBellerin': 'HectorBellerinMoruno',
                'AhmedHegazi': 'AhmedHegazy',
                'JamaalLascelles': 'JamalLascelles',
                'AngelRangel': 'AngelRangelZaragoza',
                'Zanka': 'MathiasJattahNjieJorgensen',
                'ChrisLwe': 'ChrisLowe',
                'EricBailly': 'EricBertrandBailly',
                'MarcosRojo': 'FaustinoMarcosAlbertoRojo',
                'CdricSoares': 'CedricRicardoAlvesSoares',
                'AngeloOgbonna': 'AngeloObinzeOgbonna',
                'HctorBellern': 'HectorBellerinMoruno',
                'DavinsonSanchez': 'DavinsonSanchezMina',
                'JavierManquillo': 'JavierManquilloGaitan',
                'TommySmith': 'TomSmith',
                'Bruno': 'BrunoSaltorGrau',
                'GatanBong': 'GaetanBongSongo',
                'NicolsOtamendi': 'NicolasHernanOtamendi',
                'CsarAzpilicueta': 'CesarAzpilicuetaTanco',
                'AntonioRdiger': 'AntonioRudiger',
                'JosHolebas': 'JoseHolebas',
                'SamusColeman': 'SeamusColeman',
                'AllanRomoNyom': 'AllanRomeoNyom',
                'NathanAk': 'NathanAke',
                'JosephGomez': 'JoeGomez',
                'AlbertoMoreno':'AlbertoMorenoPerez',
                'LuisAntonioValencia':'LuisAntonioValenciaMosquera',
                'VictorLindelf':'VictorNilssonLindelof',
                'DavinsonSnchez':'DavinsonSanchezMina',
                'NicolasOtamendi':'NicolasHernanOtamendi',
                'NachoMonreal':'IgnacioMonrealEraso',
                'FedericoFernndez':'FedericoFernandez',
                'SebastianPrdl':'SebastianProdl',
                'CedricSoares':'CedricRicardoAlvesSoares',
                'JoelMatip':'JoelAndreJobMatip',
                'MiguelBritos':'MiguelAngelBritosCabrera',
                'VictorLindelof':'VictorNilssonLindelof',
                'JamesCollins':'JamesMichaelCollins',
                'CucoMartina':'RhuendlyMartina',
                'DavidLuiz':'DavidLuizMoreiraMarinho',
                'MollaWagu':'MollaWague',
                'JrmyPied':'JeremyPied',
                'ChancelMbemba':'ChancelMbembaMangulu',
                'PabloZabaleta':'PabloJavierZabaletaGirod',
                'KikoFemenia':'FranciscoFemeniaFar',
                'CheikhouKouyat':'CheikhouKouyate',
                'KikoFemena':'FranciscoFemeniaFar',
                'JoseFonte':'JoseMigueldaRochaFonte',
                'JosFonte':'JoseMigueldaRochaFonte',
                'JesusGamez':'JesusGamezDuarte'}


**Creating a metrics collection function that takes in x (match_id) and y (player name) and returns the standard metrics-**

In [106]:
def getmetrics(x, y):
    try:
        y = player_map[y]
    except:
        pass
    split_y = re.findall('[A-Z][^A-Z]*', y)
    try:
        pass_df = vaep_values_pass.loc[
            (vaep_values_pass['name'].str.contains(split_y[-1]))
            & (vaep_values_pass['name'].str.contains(split_y[-2])) &
            (vaep_values_pass['game_id'] == int(x))]
    except:
        pass_df = vaep_values_pass.loc[
            (vaep_values_pass['name'].str.contains(split_y[-1]))
            & (vaep_values_pass['game_id'] == int(x))]
    numpasses = len(pass_df)
    numaccpasses = len(pass_df.loc[pass_df['result_name'] == 'success'])

    a = pass_df.loc[pass_df['result_name'] == 'success']['start_x'].tolist()
    b = pass_df.loc[pass_df['result_name'] == 'success']['start_y'].tolist()
    c = pass_df.loc[pass_df['result_name'] == 'success']['end_x'].tolist()
    d = pass_df.loc[pass_df['result_name'] == 'success']['end_y'].tolist()
    accpasslocs = [[[a[i], b[i]], [c[i], d[i]]] for i in range(len(a))]

    a = pass_df.loc[pass_df['result_name'] == 'fail']['start_x'].tolist()
    b = pass_df.loc[pass_df['result_name'] == 'fail']['start_y'].tolist()
    c = pass_df.loc[pass_df['result_name'] == 'fail']['end_x'].tolist()
    d = pass_df.loc[pass_df['result_name'] == 'fail']['end_y'].tolist()
    inaccpasslocs = [[[a[i], b[i]], [c[i], d[i]]] for i in range(len(a))]

    acc_vaep_values = pass_df.loc[pass_df['result_name'] ==
                                  'success']['vaep_value'].tolist()
    inacc_vaep_values = pass_df.loc[pass_df['result_name'] ==
                                    'fail']['vaep_value'].tolist()
    acc_off_values = pass_df.loc[pass_df['result_name'] ==
                                 'success']['offensive_value'].tolist()
    inacc_off_values = pass_df.loc[pass_df['result_name'] ==
                                   'fail']['offensive_value'].tolist()
    acc_def_values = pass_df.loc[pass_df['result_name'] ==
                                 'success']['defensive_value'].tolist()
    inacc_def_values = pass_df.loc[pass_df['result_name'] ==
                                   'fail']['defensive_value'].tolist()
    return [
        numpasses, numaccpasses, accpasslocs, inaccpasslocs, acc_vaep_values,
        inacc_vaep_values, acc_off_values, inacc_off_values, acc_def_values,
        inacc_def_values
    ]

In [3]:
# getmetrics(2500081,"Bruno")

In [101]:
new_cols = ['RB_all',
            'R_CB_all',
            'L_CB_all',
            'LB_all',
            'RCB_all',
            'CB_all',
            'LCB_all',
            'RWB_all',
            'LWB_all']

**Collecting metrics for each defender location for various clusters**

In [108]:
#R_CB - Right center back for 4 defender formation
#RCB - Right center back for 3 or 5 defender formation
#L_CB - Left center back for 4 defender formation
#LCB - Left center back for 3 or 5 defender formation
df_clusters_updated = list()
for df in tqdm(df_clusters):
    df = df.reindex(columns = df.columns.tolist() + new_cols)
    if df.iloc[0]['backline'] == 4.0:     
        df['RB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RB), axis=1)
        df['R_CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x['R_CB']), axis=1)
        df['L_CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x['L_CB']), axis=1)
        df['LB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LB), axis=1)
        df_clusters_updated.append(df)
    
    elif df.iloc[0]['backline'] == 3.0:
        df['RCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RCB), axis=1)
        df['CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.CB), axis=1)
        df['LCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LCB), axis=1)
        df_clusters_updated.append(df)
        
    elif df.iloc[0]['backline'] == 5.0:
        df['RWB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RWB), axis=1)
        df['RCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.RCB), axis=1)
        df['CB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.CB), axis=1)
        df['LCB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LCB), axis=1)
        df['LWB_all'] = df.apply(lambda x: getmetrics(x.wyId,x.LWB), axis=1)
        df_clusters_updated.append(df)


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [19:58<00:00, 92.22s/it]


**Splitting the metrics into individual columns**

In [109]:
df_clusters_metrics = list()
for df in tqdm(df_clusters_updated):
    if df.iloc[0]['backline'] == 4.0:
        df[[
            'RB_pass', 'RB_accpass', 'RB_accpassloc', 'RB_inaccpassloc',
            'RB_accvaep', 'RB_inaccvaep', 'RB_accoff', 'RB_inaccoff',
            'RB_accdef', 'RB_inaccdef'
        ]] = pd.DataFrame(df['RB_all'].to_list(), index=df.index)
        df[[
            'R_CB_pass', 'R_CB_accpass', 'R_CB_accpassloc',
            'R_CB_inaccpassloc', 'R_CB_accvaep', 'R_CB_inaccvaep',
            'R_CB_accoff', 'R_CB_inaccoff', 'R_CB_accdef', 'R_CB_inaccdef'
        ]] = pd.DataFrame(df['R_CB_all'].to_list(), index=df.index)
        df[[
            'L_CB_pass', 'L_CB_accpass', 'L_CB_accpassloc', 'L_CB_inaccpassloc',
            'L_CB_accvaep', 'L_CB_inaccvaep', 'L_CB_accoff', 'L_CB_inaccoff',
            'L_CB_accdef', 'L_CB_inaccdef'
        ]] = pd.DataFrame(df['L_CB_all'].to_list(), index=df.index)
        df[[
            'LB_pass', 'LB_accpass', 'LB_accpassloc', 'LB_inaccpassloc',
            'LB_accvaep', 'LB_inaccvaep', 'LB_accoff', 'LB_inaccoff',
            'LB_accdef', 'LB_inaccdef'
        ]] = pd.DataFrame(df['LB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)

    elif df.iloc[0]['backline'] == 3.0:
        df[[
            'RCB_pass', 'RCB_accpass', 'RCB_accpassloc', 'RCB_inaccpassloc',
            'RCB_accvaep', 'RCB_inaccvaep', 'RCB_accoff', 'RCB_inaccoff',
            'RCB_accdef', 'RCB_inaccdef'
        ]] = pd.DataFrame(df['RCB_all'].to_list(), index=df.index)
        df[[
            'CB_pass', 'CB_accpass', 'CB_accpassloc', 'CB_inaccpassloc',
            'CB_accvaep', 'CB_inaccvaep', 'CB_accoff', 'CB_inaccoff',
            'CB_accdef', 'CB_inaccdef'
        ]] = pd.DataFrame(df['CB_all'].to_list(), index=df.index)
        df[[
            'LCB_pass', 'LCB_accpass', 'LCB_accpassloc', 'LCB_inaccpassloc',
            'LCB_accvaep', 'LCB_inaccvaep', 'LCB_accoff', 'LCB_inaccoff',
            'LCB_accdef', 'LCB_inaccdef'
        ]] = pd.DataFrame(df['LCB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)

    elif df.iloc[0]['backline'] == 5.0:
        df[[
            'RCB_pass', 'RCB_accpass', 'RCB_accpassloc', 'RCB_inaccpassloc',
            'RCB_accvaep', 'RCB_inaccvaep', 'RCB_accoff', 'RCB_inaccoff',
            'RCB_accdef', 'RCB_inaccdef'
        ]] = pd.DataFrame(df['RCB_all'].to_list(), index=df.index)
        df[[
            'CB_pass', 'CB_accpass', 'CB_accpassloc', 'CB_inaccpassloc',
            'CB_accvaep', 'CB_inaccvaep', 'CB_accoff', 'CB_inaccoff',
            'CB_accdef', 'CB_inaccdef'
        ]] = pd.DataFrame(df['CB_all'].to_list(), index=df.index)
        df[[
            'LCB_pass', 'LCB_accpass', 'LCB_accpassloc', 'LCB_inaccpassloc',
            'LCB_accvaep', 'LCB_inaccvaep', 'LCB_accoff', 'LCB_inaccoff',
            'LCB_accdef', 'LCB_inaccdef'
        ]] = pd.DataFrame(df['LCB_all'].to_list(), index=df.index)
        df[[
            'RWB_pass', 'RWB_accpass', 'RWB_accpassloc', 'RWB_inaccpassloc',
            'RWB_accvaep', 'RWB_inaccvaep', 'RWB_accoff', 'RWB_inaccoff',
            'RWB_accdef', 'RWB_inaccdef'
        ]] = pd.DataFrame(df['RWB_all'].to_list(), index=df.index)
        df[[
            'LWB_pass', 'LWB_accpass', 'LWB_accpassloc', 'LWB_inaccpassloc',
            'LWB_accvaep', 'LWB_inaccvaep', 'LWB_accoff', 'LWB_inaccoff',
            'LWB_accdef', 'LWB_inaccdef'
        ]] = pd.DataFrame(df['LWB_all'].to_list(), index=df.index)
        df.drop([
            'RB_all', 'R_CB_all', 'L_CB_all', 'LB_all', 'RCB_all', 'LCB_all',
            'CB_all', 'RWB_all', 'LWB_all'
        ],
                axis=1,
                inplace=True)
        df_clusters_metrics.append(df)


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 26.32it/s]


In [110]:
names = ['rrrl','rrll','rrl','rrr','rll','rrrll','rrlr','rrrr','rrrrl','rlr','rrrlr','rrlll','rlll']

for i,df in enumerate(df_clusters_metrics):
    df.to_pickle(f'../../data/clusters/clusters_vaep/cluster_{names[i]}.pkl')

In [111]:
rrrl = pd.read_pickle(f'../../data/clusters/clusters_vaep/cluster_rrrl.pkl')
rrrl

,wyId,team,team_defense,RB,R_CB,L_CB,LB,RCB,CB,LCB,RWB,LWB,backline,match,gameweek,teamsData,dateutc,venue,referees,score,footedness,temp,RB_pass,RB_accpass,RB_accpassloc,RB_inaccpassloc,RB_accvaep,RB_inaccvaep,RB_accoff,RB_inaccoff,RB_accdef,RB_inaccdef,R_CB_pass,R_CB_accpass,R_CB_accpassloc,R_CB_inaccpassloc,R_CB_accvaep,R_CB_inaccvaep,R_CB_accoff,R_CB_inaccoff,R_CB_accdef,R_CB_inaccdef,L_CB_pass,L_CB_accpass,L_CB_accpassloc,L_CB_inaccpassloc,L_CB_accvaep,L_CB_inaccvaep,L_CB_accoff,L_CB_inaccoff,L_CB_accdef,L_CB_inaccdef,LB_pass,LB_accpass,LB_accpassloc,LB_inaccpassloc,LB_accvaep,LB_inaccvaep,LB_accoff,LB_inaccoff,LB_accdef,LB_inaccdef
0,2499719,Leicester,"[DannySimpson, WesMorgan, HarryMaguire, Christ...",DannySimpson,WesMorgan,HarryMaguire,ChristianFuchs,NaN,NaN,NaN,NaN,NaN,4,Arsenal-Leicester,1,"{'1609': {'scoreET': 0, 'coachId': 7845, 'side...",2017-08-11 18:45:00,Emirates Stadium,"[{'refereeId': 385909, 'role': 'referee'}, {'r...",4 - 3,right-right-right-left,2499719Leicester,28,17,"[[[74.55, 25.159999999999997], [77.7, 2.040000...","[[[14.700000000000003, 62.56], [92.4, 0.0]], [...","[0.0016304564196616411, -0.002541720401495695,...","[-0.007993863895535469, -0.0038780230097472668...","[0.0009762709960341454, -0.003303314559161663,...","[-0.00800476223230362, -0.0031672222539782524,...","[0.0006541854236274958, 0.0007615942740812898,...","[1.0898802429437637e-05, -0.000710800755769014...",17,16,"[[[55.65, 36.04], [45.15, 34.68]], [[53.55, 8....","[[[81.9, 26.520000000000003], [49.35, 23.11999...","[0.0025919945910573006, 0.0015028445050120354,...",[0.004824420902878046],"[0.002497009001672268, 0.0016154823824763298, ...",[-4.612840712070465e-05],"[9.498558938503265e-05, -0.0001126378774642944...",[0.004870549309998751],22,21,"[[[80.85, 17.0], [64.05, 10.200000000000003]],...","[[[65.1, 28.560000000000002], [27.299999999999...","[0.008949296548962593, 0.00964261218905449, 0....",[-0.0022838639561086893],"[0.0022858676966279745, 0.0030200097244232893,...",[-0.0016686758026480675],"[0.006663428619503975, 0.0066226026974618435, ...",[-0.0006151881534606218],34,19,"[[[9.450000000000003, 4.079999999999998], [24....","[[[80.85, 3.4000000000000057], [38.84999999999...","[-0.00010392419062554836, 0.003356457687914371...","[-0.002728058258071542, -0.003620159113779664,...","[-0.003604849800467491, 0.000636824406683445, ...","[-0.0014283563941717148, -0.002075143158435821...","[0.003500925609841943, 0.0027196332812309265, ...","[-0.001299701863899827, -0.0015450159553438425..."
2,2499720,Brighton,"[Bruno, ShaneDuffy, LewisDunk, MarkusSuttner]",Bruno,ShaneDuffy,LewisDunk,MarkusSuttner,NaN,NaN,NaN,NaN,NaN,4,Brighton-Man City,1,"{'1651': {'scoreET': 0, 'coachId': 8093, 'side...",2017-08-12 16:30:00,The American Express Community Stadium,"[{'refereeId': 384965, 'role': 'referee'}, {'r...",0 - 2,right-right-right-left,2499720Brighton,24,16,"[[[66.15, 5.44], [66.15, 17.0]], [[35.7, 25.84...","[[[34.65, 13.6], [55.65, 34.68]], [[35.7, 2.72...","[0.0026282663457095623, -0.0002443061675876379...","[-0.007042461074888706, -0.000629188260063529,...","[0.0022309990599751472, 0.000527428463101387, ...","[-0.0030955024994909763, 3.343075513839722e-05...","[0.0003972674021497369, -0.0007717346306890249...","[-0.00394695857539773, -0.0006626190152019262,...",11,8,"[[[35.7, 38.76], [35.7, 62.56]], [[29.4, 38.76...","[[[19.95, 30.6], [38.85, 65.28]], [[24.15, 18....","[0.0002864398993551731, 0.0027896929532289505,...","[0.021345699205994606, -0.009991684928536415, ...","[0.00020324718207120895, 0.0011601513251662254...","[0.0012216665782034397, 0.00285534979775548, -...","[8.319271728396416e-05, 0.001629541628062725, ...","[0.020124033093452454, -0.012847034260630608, ...",10,7,"[[[14.7, 58.48], [8.4, 36.72]], [[23.1, 63.24]...","[[[23.1, 46.92], [59.85, 34.0]]]","[-0.00158714409917593, 0.0033819382078945637, ...",[0.007757215760648251],"[-0.00041411444544792175, 0.000827418873086571...",[0.0003146685194224119],"[-0.0011730296537280083, 0.00255451